In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np 

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
reset_graph()

NameError: name 'tf' is not defined

In [3]:
import tensorflow as tf

n_inputs = 28 * 28
n_hidden = 100
n_outputs = 5

In [20]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

training = tf.placeholder_with_default(False, shape=(), name='training')

he_init = tf.contrib.layers.variance_scaling_initializer()

hidden1 = tf.layers.dense(X, n_hidden, name='hidden1', activation=tf.nn.elu, 
                          kernel_initializer=he_init)
hidden2 = tf.layers.dense(hidden1, n_hidden, name='hidden2', activation=tf.nn.elu, 
                          kernel_initializer=he_init)
hidden3 = tf.layers.dense(hidden2, n_hidden, name='hidden3', 
                          activation=tf.nn.elu, kernel_initializer=he_init)
hidden4 = tf.layers.dense(hidden3, n_hidden, name='hidden4', 
                          activation=tf.nn.elu, kernel_initializer=he_init)
hidden5 = tf.layers.dense(hidden4, n_hidden, name='hidden5', 
                          activation=tf.nn.elu, kernel_initializer=he_init)

logits = tf.layers.dense(hidden4, n_outputs, name='outputs', kernel_initializer=he_init)
Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
learning_rate = 0.001
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    

In [21]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

In [7]:
from datetime import datetime

In [9]:
def train(n_epochs=1000, batch_size = 20, max_checks_without_progress=20):
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = 'tf_logs'
    logdir = "{}/run-{}/".format(root_logdir, now)

    loss_summary = tf.summary.scalar('loss', loss)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

    checks_without_progress = 0
    best_loss = np.infty
    step_counter = 0

    with tf.Session() as sess:
        init.run()

        for epoch in range(n_epochs):
            rnd_idx = np.random.permutation(len(X_train1))
            for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
                X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
                sess.run([training_op, extra_update_ops], 
                         feed_dict={ X: X_batch, y: y_batch, training: True })
                if step_counter % 10 == 0:
                    summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch})
                    step = step_counter
                    file_writer.add_summary(summary_str, step)
                step_counter += 1
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={ X: X_valid1, y: y_valid1 })
            if loss_val < best_loss:
                checks_without_progress = 0
                best_loss = loss_val
                save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            else:
                checks_without_progress += 1
                if checks_without_progress > max_checks_without_progress:
                    file_writer.close()
                    print('Early stop!')
                    break
            print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    with tf.Session() as sess:
        saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
        acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
        print("Final test accuracy: {:.2f}%".format(acc_test * 100))

In [8]:
reset_graph()

training = tf.placeholder_with_default(False, shape=(), name='training')
he_init = tf.contrib.layers.variance_scaling_initializer()

In [10]:
from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

def dnn_bn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init, with_dropout=False, dropout_rate=0.5):
    with tf.variable_scope(name, 'dnn_bn'):
        for layer in range(n_hidden_layers):
            raw_output = tf.layers.dense(inputs, n_neurons, kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
            bn_inputs = my_batch_norm_layer(raw_output)
            inputs = activation(bn_inputs)
            if with_dropout:
                inputs = tf.layers.dropout(inputs, dropout_rate, training=training)
        return inputs

In [11]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

dnn_bn_outputs = dnn_bn(X, with_dropout=True)

logits = tf.layers.dense(dnn_bn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
learning_rate = 0.001
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

train()

0	Validation loss: 0.120866	Best loss: 0.120866	Accuracy: 96.09%
1	Validation loss: 0.100900	Best loss: 0.100900	Accuracy: 96.52%
2	Validation loss: 0.094376	Best loss: 0.094376	Accuracy: 97.26%
3	Validation loss: 0.090971	Best loss: 0.090971	Accuracy: 97.15%
4	Validation loss: 0.075347	Best loss: 0.075347	Accuracy: 97.65%
5	Validation loss: 0.073190	Best loss: 0.073190	Accuracy: 97.69%
6	Validation loss: 0.070634	Best loss: 0.070634	Accuracy: 97.54%
7	Validation loss: 0.064752	Best loss: 0.064752	Accuracy: 97.73%
8	Validation loss: 0.060272	Best loss: 0.060272	Accuracy: 97.85%
9	Validation loss: 0.059940	Best loss: 0.059940	Accuracy: 98.01%
10	Validation loss: 0.054780	Best loss: 0.054780	Accuracy: 98.16%
11	Validation loss: 0.050559	Best loss: 0.050559	Accuracy: 98.36%
12	Validation loss: 0.053041	Best loss: 0.050559	Accuracy: 98.16%
13	Validation loss: 0.048494	Best loss: 0.048494	Accuracy: 98.48%
14	Validation loss: 0.048790	Best loss: 0.048494	Accuracy: 98.32%
15	Validation loss: 

In [6]:
X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

In [15]:
reset_graph()

# transfer_learning
saver = tf.train.import_meta_graph('./my_mnist_model_0_to_4.ckpt.meta')

for op in tf.get_default_graph().get_operations():
    print(op.name)

training/input
training
X
y
dnn_bn/hidden1/kernel/Initializer/truncated_normal/shape
dnn_bn/hidden1/kernel/Initializer/truncated_normal/mean
dnn_bn/hidden1/kernel/Initializer/truncated_normal/stddev
dnn_bn/hidden1/kernel/Initializer/truncated_normal/TruncatedNormal
dnn_bn/hidden1/kernel/Initializer/truncated_normal/mul
dnn_bn/hidden1/kernel/Initializer/truncated_normal
dnn_bn/hidden1/kernel
dnn_bn/hidden1/kernel/Assign
dnn_bn/hidden1/kernel/read
dnn_bn/hidden1/bias/Initializer/zeros
dnn_bn/hidden1/bias
dnn_bn/hidden1/bias/Assign
dnn_bn/hidden1/bias/read
dnn_bn/hidden1/MatMul
dnn_bn/hidden1/BiasAdd
dnn_bn/batch_normalization/beta/Initializer/zeros
dnn_bn/batch_normalization/beta
dnn_bn/batch_normalization/beta/Assign
dnn_bn/batch_normalization/beta/read
dnn_bn/batch_normalization/gamma/Initializer/ones
dnn_bn/batch_normalization/gamma
dnn_bn/batch_normalization/gamma/Assign
dnn_bn/batch_normalization/gamma/read
dnn_bn/batch_normalization/moving_mean/Initializer/zeros
dnn_bn/batch_normal

save/RestoreV2_12/shape_and_slices
save/RestoreV2_12
save/Assign_12
save/RestoreV2_13/tensor_names
save/RestoreV2_13/shape_and_slices
save/RestoreV2_13
save/Assign_13
save/RestoreV2_14/tensor_names
save/RestoreV2_14/shape_and_slices
save/RestoreV2_14
save/Assign_14
save/RestoreV2_15/tensor_names
save/RestoreV2_15/shape_and_slices
save/RestoreV2_15
save/Assign_15
save/RestoreV2_16/tensor_names
save/RestoreV2_16/shape_and_slices
save/RestoreV2_16
save/Assign_16
save/RestoreV2_17/tensor_names
save/RestoreV2_17/shape_and_slices
save/RestoreV2_17
save/Assign_17
save/RestoreV2_18/tensor_names
save/RestoreV2_18/shape_and_slices
save/RestoreV2_18
save/Assign_18
save/RestoreV2_19/tensor_names
save/RestoreV2_19/shape_and_slices
save/RestoreV2_19
save/Assign_19
save/RestoreV2_20/tensor_names
save/RestoreV2_20/shape_and_slices
save/RestoreV2_20
save/Assign_20
save/RestoreV2_21/tensor_names
save/RestoreV2_21/shape_and_slices
save/RestoreV2_21
save/Assign_21
save/RestoreV2_22/tensor_names
save/Resto

In [19]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss/loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("eval/Mean:0")

In [24]:
learning_rate = 0.01
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

In [25]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

In [20]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

In [40]:
X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [41]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    for var in output_layer_vars:
        var.initializer.run()

    t0 = time.time()
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_0_to_4.ckpt
0	Validation loss: 1.036334	Best loss: 1.036334	Accuracy: 64.67%
1	Validation loss: 1.065142	Best loss: 1.036334	Accuracy: 62.00%
2	Validation loss: 1.049612	Best loss: 1.036334	Accuracy: 59.33%
3	Validation loss: 0.981039	Best loss: 0.981039	Accuracy: 67.33%
4	Validation loss: 0.968357	Best loss: 0.968357	Accuracy: 68.67%
5	Validation loss: 0.956786	Best loss: 0.956786	Accuracy: 66.00%
6	Validation loss: 0.948053	Best loss: 0.948053	Accuracy: 66.00%
7	Validation loss: 1.005618	Best loss: 0.948053	Accuracy: 62.67%
8	Validation loss: 0.948277	Best loss: 0.948053	Accuracy: 68.00%
9	Validation loss: 0.944706	Best loss: 0.944706	Accuracy: 68.00%
10	Validation loss: 0.930088	Best loss: 0.930088	Accuracy: 67.33%
11	Validation loss: 0.967852	Best loss: 0.930088	Accuracy: 64.67%
12	Validation loss: 0.945359	Best loss: 0.930088	Accuracy: 66.00%
13	Validation loss: 0.959653	Best loss: 0.930088	Accuracy: 64.67%
14	Validation l

In [43]:
hidden5_out = tf.get_default_graph().get_tensor_by_name("dnn_bn/dropout_5/cond/Merge:0")

In [48]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    for var in output_layer_vars:
        var.initializer.run()

    t0 = time.time()
    
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_0_to_4.ckpt
0	Validation loss: 1.049223	Best loss: 1.049223	Accuracy: 62.00%
1	Validation loss: 1.035855	Best loss: 1.035855	Accuracy: 62.67%
2	Validation loss: 1.002728	Best loss: 1.002728	Accuracy: 66.00%
3	Validation loss: 0.966724	Best loss: 0.966724	Accuracy: 65.33%
4	Validation loss: 0.962987	Best loss: 0.962987	Accuracy: 66.00%
5	Validation loss: 0.964577	Best loss: 0.962987	Accuracy: 67.33%
6	Validation loss: 0.940518	Best loss: 0.940518	Accuracy: 70.00%
7	Validation loss: 0.929014	Best loss: 0.929014	Accuracy: 69.33%
8	Validation loss: 0.956908	Best loss: 0.929014	Accuracy: 69.33%
9	Validation loss: 0.934276	Best loss: 0.929014	Accuracy: 69.33%
10	Validation loss: 0.966893	Best loss: 0.929014	Accuracy: 62.00%
11	Validation loss: 0.943480	Best loss: 0.929014	Accuracy: 70.67%
12	Validation loss: 0.928534	Best loss: 0.928534	Accuracy: 70.00%
13	Validation loss: 0.927442	Best loss: 0.927442	Accuracy: 67.33%
14	Validation l

In [51]:
reset_graph()

n_outputs = 5

restore_saver = tf.train.import_meta_graph("./my_mnist_model_0_to_4.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden4_out = tf.get_default_graph().get_tensor_by_name("dnn_bn/dropout_4/cond/Merge:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [52]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [54]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./my_mnist_model_5_to_9_four_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_0_to_4.ckpt
0	Validation loss: 0.982947	Best loss: 0.982947	Accuracy: 68.67%
1	Validation loss: 1.034599	Best loss: 0.982947	Accuracy: 65.33%
2	Validation loss: 0.990594	Best loss: 0.982947	Accuracy: 62.67%
3	Validation loss: 0.927790	Best loss: 0.927790	Accuracy: 67.33%
4	Validation loss: 0.973629	Best loss: 0.927790	Accuracy: 65.33%
5	Validation loss: 0.917403	Best loss: 0.917403	Accuracy: 68.00%
6	Validation loss: 0.900404	Best loss: 0.900404	Accuracy: 70.00%
7	Validation loss: 0.909795	Best loss: 0.900404	Accuracy: 71.33%
8	Validation loss: 0.892049	Best loss: 0.892049	Accuracy: 70.67%
9	Validation loss: 0.898367	Best loss: 0.892049	Accuracy: 70.00%
10	Validation loss: 0.908349	Best loss: 0.892049	Accuracy: 69.33%
11	Validation loss: 0.930747	Best loss: 0.892049	Accuracy: 68.00%
12	Validation loss: 0.890273	Best loss: 0.890273	Accuracy: 71.33%
13	Validation loss: 0.874466	Best loss: 0.874466	Accuracy: 72.67%
14	Validation l

Early stopping!
INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_four_frozen
Final test accuracy: 69.84%


In [55]:
learning_rate = 0.01

unfrozen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="dnn_bn/dropout_[34]/cond/Merge|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam3")
training_op = optimizer.minimize(loss, var_list=unfrozen_vars)

init = tf.global_variables_initializer()
two_frozen_saver = tf.train.Saver()

In [56]:

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = two_frozen_saver.save(sess, "./my_mnist_model_5_to_9_two_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_four_frozen
0	Validation loss: 0.825613	Best loss: 0.825613	Accuracy: 76.00%
1	Validation loss: 0.805913	Best loss: 0.805913	Accuracy: 78.00%
2	Validation loss: 0.835217	Best loss: 0.805913	Accuracy: 73.33%
3	Validation loss: 0.834071	Best loss: 0.805913	Accuracy: 74.00%
4	Validation loss: 0.813341	Best loss: 0.805913	Accuracy: 77.33%
5	Validation loss: 0.816464	Best loss: 0.805913	Accuracy: 75.33%
6	Validation loss: 0.830536	Best loss: 0.805913	Accuracy: 77.33%
7	Validation loss: 0.812285	Best loss: 0.805913	Accuracy: 77.33%
8	Validation loss: 0.834345	Best loss: 0.805913	Accuracy: 76.67%
9	Validation loss: 0.840161	Best loss: 0.805913	Accuracy: 78.00%
10	Validation loss: 0.814254	Best loss: 0.805913	Accuracy: 77.33%
11	Validation loss: 0.850210	Best loss: 0.805913	Accuracy: 77.33%
12	Validation loss: 0.800454	Best loss: 0.800454	Accuracy: 80.00%
13	Validation loss: 0.813327	Best loss: 0.800454	Accuracy: 76.00%
14	Valid

In [57]:
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam4")
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
no_frozen_saver = tf.train.Saver()

In [9]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = no_frozen_saver.save(sess, "./my_mnist_model_5_to_9_no_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    no_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_no_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

NameError: name 'init' is not defined

In [12]:
from dnn_classifier import DNNClassifier
reset_graph()

dnn_clf = DNNClassifier(random_state=43)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

2	Validation loss: 0.209017	Best loss: 0.105918	Accuracy: 95.93%
3	Validation loss: 0.112292	Best loss: 0.105918	Accuracy: 97.65%
4	Validation loss: 0.157204	Best loss: 0.105918	Accuracy: 96.44%
5	Validation loss: 0.146383	Best loss: 0.105918	Accuracy: 97.11%
7	Validation loss: 0.140665	Best loss: 0.092989	Accuracy: 97.30%
8	Validation loss: 0.124650	Best loss: 0.092989	Accuracy: 97.73%
9	Validation loss: 0.135034	Best loss: 0.092989	Accuracy: 98.16%
10	Validation loss: 0.356990	Best loss: 0.092989	Accuracy: 93.59%
11	Validation loss: 0.221628	Best loss: 0.092989	Accuracy: 97.46%
12	Validation loss: 0.241587	Best loss: 0.092989	Accuracy: 94.72%
13	Validation loss: 1.897409	Best loss: 0.092989	Accuracy: 81.39%
14	Validation loss: 0.174883	Best loss: 0.092989	Accuracy: 96.91%
15	Validation loss: 0.971001	Best loss: 0.092989	Accuracy: 58.87%
16	Validation loss: 7.579824	Best loss: 0.092989	Accuracy: 60.75%
17	Validation loss: 1.654402	Best loss: 0.092989	Accuracy: 60.05%
18	Validation los

DNNClassifier(activation=<function elu at 0x108659f28>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x11c372e18>,
       learning_rate=0.01, n_hidden_layers=5, n_nuerons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=43)

In [13]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.98268145553609654

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_nuerons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                fit_params={"X_valid": X_valid1, "y_valid": y_valid1, "n_epochs": 1000},
                                random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_nuerons=10, learning_rate=0.05, batch_size=100, activation=<function elu at 0x108659f28> 
1	Validation loss: 0.153023	Best loss: 0.133020	Accuracy: 95.78%
2	Validation loss: 0.137756	Best loss: 0.133020	Accuracy: 96.44%
4	Validation loss: 0.134245	Best loss: 0.113295	Accuracy: 96.68%
5	Validation loss: 0.160136	Best loss: 0.113295	Accuracy: 96.48%
6	Validation loss: 1.205526	Best loss: 0.113295	Accuracy: 58.29%
7	Validation loss: 0.854387	Best loss: 0.113295	Accuracy: 58.44%
8	Validation loss: 1.618615	Best loss: 0.113295	Accuracy: 19.27%
9	Validation loss: 1.610767	Best loss: 0.113295	Accuracy: 22.01%
10	Validation loss: 1.613220	Best loss: 0.113295	Accuracy: 19.27%
11	Validation loss: 1.611311	Best loss: 0.113295	Accuracy: 22.01%
12	Validation loss: 1.627571	Best loss: 0.113295	Accuracy: 22.01%
13	Validation loss: 1.642730	Best loss: 0.113295	Accuracy: 18.73%
14	Validation loss: 1.615385	Best loss: 0.113295	Accuracy

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.2s remaining:    0.0s


2	Validation loss: 0.117152	Best loss: 0.108809	Accuracy: 96.99%
3	Validation loss: 0.147208	Best loss: 0.108809	Accuracy: 96.52%
4	Validation loss: 0.140152	Best loss: 0.108809	Accuracy: 96.36%
5	Validation loss: 0.141784	Best loss: 0.108809	Accuracy: 96.44%
6	Validation loss: 1.561300	Best loss: 0.108809	Accuracy: 39.44%
7	Validation loss: 1.164626	Best loss: 0.108809	Accuracy: 39.80%
8	Validation loss: 1.150888	Best loss: 0.108809	Accuracy: 40.15%
9	Validation loss: 1.150942	Best loss: 0.108809	Accuracy: 40.15%
10	Validation loss: 1.135206	Best loss: 0.108809	Accuracy: 42.49%
11	Validation loss: 1.158850	Best loss: 0.108809	Accuracy: 40.23%
12	Validation loss: 1.142637	Best loss: 0.108809	Accuracy: 42.34%
13	Validation loss: 1.173786	Best loss: 0.108809	Accuracy: 40.27%
14	Validation loss: 1.146071	Best loss: 0.108809	Accuracy: 42.30%
15	Validation loss: 1.679023	Best loss: 0.108809	Accuracy: 18.73%
16	Validation loss: 1.617953	Best loss: 0.108809	Accuracy: 22.01%
17	Validation loss

15	Validation loss: 0.064104	Best loss: 0.063263	Accuracy: 98.40%
16	Validation loss: 0.076903	Best loss: 0.063263	Accuracy: 98.28%
17	Validation loss: 0.079097	Best loss: 0.063263	Accuracy: 97.85%
18	Validation loss: 0.068662	Best loss: 0.063263	Accuracy: 98.24%
19	Validation loss: 0.085785	Best loss: 0.063263	Accuracy: 97.97%
20	Validation loss: 0.075248	Best loss: 0.063263	Accuracy: 98.20%
21	Validation loss: 0.080703	Best loss: 0.063263	Accuracy: 98.08%
22	Validation loss: 0.069105	Best loss: 0.063263	Accuracy: 98.28%
23	Validation loss: 0.107339	Best loss: 0.063263	Accuracy: 98.16%
24	Validation loss: 0.097046	Best loss: 0.063263	Accuracy: 98.32%
25	Validation loss: 0.083067	Best loss: 0.063263	Accuracy: 98.12%
26	Validation loss: 0.103586	Best loss: 0.063263	Accuracy: 98.16%
27	Validation loss: 0.079554	Best loss: 0.063263	Accuracy: 98.44%
Early stop!
[CV]  n_nuerons=30, learning_rate=0.02, batch_size=500, activation=<function relu at 0x10866a158>, total=   7.8s
[CV] n_nuerons=90

[CV]  n_nuerons=70, learning_rate=0.1, batch_size=50, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x11e7f0158>, total=  36.8s
[CV] n_nuerons=70, learning_rate=0.1, batch_size=50, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x11e7f0158> 
2	Validation loss: 506.760010	Best loss: 306.050323	Accuracy: 80.88%
6	Validation loss: 277.744843	Best loss: 100.716995	Accuracy: 90.19%
7	Validation loss: 116.952637	Best loss: 100.716995	Accuracy: 93.04%
8	Validation loss: 212.281174	Best loss: 100.716995	Accuracy: 89.29%
9	Validation loss: 125.738800	Best loss: 100.716995	Accuracy: 94.45%
12	Validation loss: 42.503910	Best loss: 22.063108	Accuracy: 95.00%
13	Validation loss: 29.345461	Best loss: 22.063108	Accuracy: 95.82%
14	Validation loss: 46.292641	Best loss: 22.063108	Accuracy: 95.39%
15	Validation loss: 6068046.000000	Best loss: 22.063108	Accuracy: 50.59%
16	Validation loss: 60703.898438	Best loss: 22.063108	Accuracy: 94.02%
17	Validation loss: 40

...

19	Validation loss: 0.157360	Best loss: 0.071489	Accuracy: 98.16%
20	Validation loss: 0.299526	Best loss: 0.071489	Accuracy: 97.07%
21	Validation loss: 0.193950	Best loss: 0.071489	Accuracy: 97.34%
22	Validation loss: 0.144671	Best loss: 0.071489	Accuracy: 96.29%
23	Validation loss: 0.151076	Best loss: 0.071489	Accuracy: 96.87%
24	Validation loss: 0.561609	Best loss: 0.071489	Accuracy: 93.08%
Early stop!
[CV]  n_nuerons=50, learning_rate=0.02, batch_size=100, activation=<function elu at 0x108659f28>, total=  12.2s
[CV] n_nuerons=10, learning_rate=0.02, batch_size=100, activation=<function relu at 0x10866a158> 
4	Validation loss: 0.131524	Best loss: 0.117291	Accuracy: 96.33%
6	Validation loss: 0.118102	Best loss: 0.112828	Accuracy: 96.76%
7	Validation loss: 0.121050	Best loss: 0.112828	Accuracy: 97.07%
9	Validation loss: 0.129330	Best loss: 0.108198	Accuracy: 97.07%
10	Validation loss: 0.117869	Best loss: 0.108198	Accuracy: 97.22%
11	Validation loss: 0.125181	Best loss: 0.108198	Accurac

1	Validation loss: 0.487854	Best loss: 0.209483	Accuracy: 86.28%
2	Validation loss: 0.268070	Best loss: 0.209483	Accuracy: 94.37%
3	Validation loss: 0.490091	Best loss: 0.209483	Accuracy: 80.10%
4	Validation loss: 0.626691	Best loss: 0.209483	Accuracy: 71.54%
5	Validation loss: 1.589942	Best loss: 0.209483	Accuracy: 23.26%
6	Validation loss: 1.621900	Best loss: 0.209483	Accuracy: 19.12%
7	Validation loss: 1.611964	Best loss: 0.209483	Accuracy: 18.73%
8	Validation loss: 1.611319	Best loss: 0.209483	Accuracy: 22.01%
9	Validation loss: 1.608832	Best loss: 0.209483	Accuracy: 22.01%
10	Validation loss: 1.610074	Best loss: 0.209483	Accuracy: 22.01%
11	Validation loss: 1.610568	Best loss: 0.209483	Accuracy: 22.01%
12	Validation loss: 1.610718	Best loss: 0.209483	Accuracy: 19.27%
13	Validation loss: 1.611639	Best loss: 0.209483	Accuracy: 19.08%
14	Validation loss: 1.608836	Best loss: 0.209483	Accuracy: 22.01%
15	Validation loss: 1.608091	Best loss: 0.209483	Accuracy: 22.01%
16	Validation loss:

1	Validation loss: 0.298379	Best loss: 0.237627	Accuracy: 92.38%
3	Validation loss: 0.382767	Best loss: 0.199679	Accuracy: 89.05%
4	Validation loss: 0.325148	Best loss: 0.199679	Accuracy: 96.79%
5	Validation loss: 0.401838	Best loss: 0.199679	Accuracy: 89.87%
6	Validation loss: 0.452797	Best loss: 0.199679	Accuracy: 86.04%
7	Validation loss: 0.637049	Best loss: 0.199679	Accuracy: 91.13%
8	Validation loss: 0.646203	Best loss: 0.199679	Accuracy: 81.55%
9	Validation loss: 0.861209	Best loss: 0.199679	Accuracy: 52.85%
10	Validation loss: 0.958328	Best loss: 0.199679	Accuracy: 56.37%
11	Validation loss: 0.653412	Best loss: 0.199679	Accuracy: 70.05%
12	Validation loss: 0.828089	Best loss: 0.199679	Accuracy: 64.50%
13	Validation loss: 0.758362	Best loss: 0.199679	Accuracy: 66.89%
14	Validation loss: 0.338167	Best loss: 0.199679	Accuracy: 92.92%
15	Validation loss: 0.727078	Best loss: 0.199679	Accuracy: 58.84%
16	Validation loss: 1.206226	Best loss: 0.199679	Accuracy: 37.14%
17	Validation loss

8	Validation loss: 10.427986	Best loss: 9.348626	Accuracy: 93.82%
10	Validation loss: 10.168479	Best loss: 6.103171	Accuracy: 95.19%
12	Validation loss: 8.413948	Best loss: 4.817612	Accuracy: 95.19%
13	Validation loss: 9.179695	Best loss: 4.817612	Accuracy: 94.68%
14	Validation loss: 9.877934	Best loss: 4.817612	Accuracy: 93.59%
15	Validation loss: 6.845265	Best loss: 4.817612	Accuracy: 96.36%
16	Validation loss: 5.090184	Best loss: 4.817612	Accuracy: 95.97%
17	Validation loss: 4.890172	Best loss: 4.817612	Accuracy: 96.25%
20	Validation loss: 3.590352	Best loss: 3.512105	Accuracy: 96.95%
21	Validation loss: 4.379238	Best loss: 3.512105	Accuracy: 96.40%
22	Validation loss: 3.547738	Best loss: 3.512105	Accuracy: 97.15%
24	Validation loss: 2.595516	Best loss: 2.498681	Accuracy: 96.99%
25	Validation loss: 4.806193	Best loss: 2.498681	Accuracy: 95.54%
26	Validation loss: 3.438840	Best loss: 2.498681	Accuracy: 97.07%
27	Validation loss: 3.633664	Best loss: 2.498681	Accuracy: 96.95%
30	Valida

18	Validation loss: 0.089821	Best loss: 0.055529	Accuracy: 98.75%
19	Validation loss: 0.078961	Best loss: 0.055529	Accuracy: 98.79%
20	Validation loss: 0.091639	Best loss: 0.055529	Accuracy: 98.55%
21	Validation loss: 0.067926	Best loss: 0.055529	Accuracy: 98.79%
22	Validation loss: 0.107766	Best loss: 0.055529	Accuracy: 98.40%
23	Validation loss: 0.085206	Best loss: 0.055529	Accuracy: 98.87%
24	Validation loss: 0.123297	Best loss: 0.055529	Accuracy: 98.67%
25	Validation loss: 0.083747	Best loss: 0.055529	Accuracy: 98.91%
26	Validation loss: 0.099444	Best loss: 0.055529	Accuracy: 98.67%
27	Validation loss: 0.082378	Best loss: 0.055529	Accuracy: 98.71%
28	Validation loss: 0.070440	Best loss: 0.055529	Accuracy: 98.79%
29	Validation loss: 0.064048	Best loss: 0.055529	Accuracy: 98.59%
Early stop!
[CV]  n_nuerons=140, learning_rate=0.01, batch_size=500, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x11e7f0158>, total= 1.5min
[CV] n_nuerons=140, learning_rate=0.01, bat